# Take Home Assignment – Vorhersage von Festgeldabschlüssen

Ziel ist die Priorisierung von Kunden nach Abschlusswahrscheinlichkeit für Festgelder,
um Kampagnenressourcen effizient einzusetzen.

## Problemstellung
Im Rahmen einer Marketingkampagne wurden Kunden telefonisch kontaktiert, um
Festgeldprodukte zu platzieren. Ziel ist die Vorhersage, ob ein Kunde ein
Festgeld abschließt (`y = yes/no`).

Die Vorhersage soll **vor** dem Kundenkontakt erfolgen, um Kampagnen gezielt
steuern zu können.

## Datensatz
Verwendet wird der Datensatz `bank-additional-full` aus dem UCI Machine Learning Repository.

Der Datensatz enthält neben Kunden- und Kampagnenmerkmalen auch makroökonomische
Variablen (z.B. Zinsniveau, Beschäftigung), welche das Marktumfeld widerspiegeln.

In [28]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))

from src.config import (
    RAW_DATA_DIR, TARGET_COL, POSITIVE_CLASS, DROP_COLS, RANDOM_STATE
)

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)

Hinweis: Beim lokalen Handling der CSV können durch Öffnen in Excel vereinzelte Werte in `emp.var.rate`
als Datum interpretiert worden sein. Diese Artefakte werden vor der Analyse zurück auf numerische Werte gemappt.

In [29]:
# Pfad:
# data/raw/bank-additional-full.csv
data_path = RAW_DATA_DIR / "bank-additional-full.csv"

if not data_path.exists():
    raise FileNotFoundError(
        f"Datei nicht gefunden: {data_path}\n"
    )

# CSV einlesen (low_memory=False um Dtype-Warnungen durch chunkweises Einlesen zu vermeiden)
df = pd.read_csv(data_path, sep=";", low_memory=False)

# Datenqualitätsproblem beheben: In 'emp.var.rate' stehen vereinzelt Excel-Datumswerte ("01. Jan", "01. Apr"),
# vermutlich weil z.B. 1.1 bzw. 1.4 beim Öffnen der CSV in Excel als Datum interpretiert wurden.

# emp.var.rate als String normalisieren (entfernt versteckte Leerzeichen etc.)
s = df["emp.var.rate"].astype(str)

# Whitespace trimmen (auch Tabs etc.)
s = s.str.strip()

# Non-breaking space entfernen (kommt manchmal aus Excel/Copy&Paste)
s = s.str.replace("\u00A0", " ", regex=False).str.strip()

# Diese Artefakte werden hier auf die plausiblen numerischen Werte zurückgemappt.
s = s.replace(
    {
        r"^01\.\s*Jan.*$": "1.1",
        r"^01\.\s*Apr.*$": "1.4",
    },
    regex=True,
)

df["emp.var.rate"] = s

# Safety-Check: Sind noch nicht-numerische Werte da?
bad = pd.to_numeric(df["emp.var.rate"], errors="coerce").isna()
if bad.any():
    print("Noch nicht-numerische Werte in emp.var.rate (erste 20):")
    print(df.loc[bad, "emp.var.rate"].value_counts().head(20))
    raise ValueError("emp.var.rate enthält weiterhin nicht-numerische Werte. Siehe Ausgabe oben.")

# Jetzt strikt numerisch casten
df["emp.var.rate"] = pd.to_numeric(df["emp.var.rate"], errors="raise")

df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Explorative Datenanalyse (EDA)
In diesem Abschnitt werden Zielverteilung, Datenqualität sowie erste Zusammenhänge
zwischen Features und Zielvariable untersucht.

In [30]:
print("Shape:", df.shape)
print("Target distribution:\n", df[TARGET_COL].value_counts(dropna=False))
df.info() 

Shape: (41188, 21)
Target distribution:
 y
no     36548
yes     4640
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-

In [31]:
target_counts = df[TARGET_COL].value_counts()
pos_rate = (df[TARGET_COL] == POSITIVE_CLASS).mean()

display(target_counts)
print(f"Positive rate ({POSITIVE_CLASS}): {pos_rate:.3%}")

y
no     36548
yes     4640
Name: count, dtype: int64

Positive rate (yes): 11.265%


**Hinweis zur Zielverteilung:**  
Nur **11,3 %** der Kunden schließen im Datensatz ein Festgeldprodukt ab.
Die Zielvariable ist damit deutlich unausgeglichen (Class Imbalance).

Für die Modellierung bedeutet dies:
- Verwendung geeigneter Evaluationsmetriken (insbesondere **PR-AUC**),
- Fokus auf **Top-K-Metriken** zur realistischen Abbildung von Kampagnenpriorisierung,
- Berücksichtigung der Klassenungleichverteilung bei der Modellwahl
  (z.B. durch `class_weight="balanced"`).

In [32]:
# In diesem Datensatz sind fehlende Werte oft als "unknown" kodiert
unknown_counts = (df == "unknown").sum().sort_values(ascending=False)
unknown_counts = unknown_counts[unknown_counts > 0]

unknown_counts.head(20)

default      8597
education    1731
housing       990
loan          990
job           330
marital        80
dtype: int64

In [33]:
# Analyse der Zielvariable nach 'default':
pd.crosstab(df["default"], df["y"], normalize="index")

y,no,yes
default,,
no,0.87121,0.12879
unknown,0.94847,0.05153
yes,1.00000,0.00000


Hinweis: Die Abschlussrate ist bei Kunden ohne Zahlungsausfall ('no') am höchsten (~12.9 %).
Kunden mit unbekanntem Status ('unknown') weisen eine deutlich niedrigere Rate (~5.2 %) auf,
was zeigt, dass 'unknown' kein neutraler oder zufälliger Wert ist.
Bei Kunden mit bekanntem Zahlungsausfall ('yes') treten praktisch keine Abschlüsse auf.
=> Die Variable 'default' ist hoch prädiktiv und 'unknown' trägt relevante Information.

## Feature-Selektion und Data Leakage
Das Feature `duration` (Dauer des Telefonats) wird bewusst aus der Modellierung
ausgeschlossen, da es erst nach dem Kundenkontakt bekannt ist und somit zu
Data Leakage führen würde.

In [34]:
# Data Leakage: duration ist erst nach dem Call bekannt
cols_to_drop = [c for c in DROP_COLS if c in df.columns]
print("Dropping columns:", cols_to_drop)

df_model = df.drop(columns=cols_to_drop).copy()
df_model.head()

Dropping columns: ['duration']


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Modellierung und Evaluationsmetriken
Zur Modellierung werden eine interpretierbare Baseline (logistische Regression)
sowie ein leistungsfähigeres Modell (Gradient Boosting) verwendet.

Aufgrund der unausgeglichenen Zielvariable liegt der Fokus auf PR-AUC sowie
Top-K-Metriken zur realistischen Abbildung von Kampagnenpriorisierung.

### Wahl der Evaluationsmetriken

Ziel der Analyse ist nicht primär eine binäre Entscheidung („ja/nein“),
sondern die **Priorisierung von Kunden** nach ihrer Abschlusswahrscheinlichkeit,
um begrenzte Marketingressourcen effizient einzusetzen.

- **Precision@10 %** misst die Abschlussquote unter den vom Modell am höchsten
  priorisierten 10 % der Kunden. Diese Metrik ist direkt geschäftsrelevant,
  da Kampagnen typischerweise nur einen Teil der Kunden adressieren können.

- **PR-AUC (Precision-Recall-AUC)** bewertet die Ranking-Qualität mit Fokus auf
  die positive Klasse („Abschluss“) und ist bei unausgeglichenen Zielvariablen
  aussagekräftiger als Accuracy oder ROC-AUC.

- **ROC-AUC** wird ergänzend berichtet, da sie eine schwellenwert-unabhängige
  Trennschärfe des Modells beschreibt, jedoch bei starkem Klassenungleichgewicht
  weniger direkt geschäftsnah ist.

Klassische Metriken wie Accuracy sind in diesem Kontext ungeeignet, da ein Modell
auch bei hoher Accuracy viele potenzielle Abschlüsse übersehen kann.

In [35]:
X = df_model.drop(columns=[TARGET_COL])
y = df_model[TARGET_COL].astype(str)

print("X shape:", X.shape)
print("y distribution:\n", y.value_counts()) 

X shape: (41188, 19)
y distribution:
 y
no     36548
yes     4640
Name: count, dtype: int64


### Train/Test Split
Der Datensatz ist zeitlich sortiert. Um eine realistische Bewertung zu ermöglichen,
erfolgt die Trennung in Trainings- und Testdaten zeitbasiert.

In [36]:
# Time-based split: letzte 20 % als Testdaten
split_index = int(len(df_model) * 0.8)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]

y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

print("Train size:", X_train.shape)
print("Test size :", X_test.shape)
print("\nTrain target distribution:\n", y_train.value_counts(normalize=True))
print("\nTest target distribution:\n", y_test.value_counts(normalize=True))


Train size: (32950, 19)
Test size : (8238, 19)

Train target distribution:
 y
no     0.936267
yes    0.063733
Name: proportion, dtype: float64

Test target distribution:
 y
no     0.691673
yes    0.308327
Name: proportion, dtype: float64


Hinweis: Zwischen Trainings- und Testdaten zeigt sich ein signifikanter 
Unterschied in der Abschlussquote (6.4% bei den Trainingdaten und 30.8% 
bei den Testdaten). Dieser Prior Shift deutet auf zeitliche Veränderungen 
der Kampagnenstrategie sowie des Marktumfelds hin und unterstreicht die 
Relevanz einer zeitbasierten Evaluation.

In [37]:
# Abschlussquote über die Zeit (Proxy: Monatsverteilung)
df_model["is_positive"] = (df_model[TARGET_COL] == POSITIVE_CLASS).astype(int)
df_model.groupby("month")["is_positive"].mean().sort_values()

month
may    0.064347
jul    0.090466
nov    0.101439
jun    0.105115
aug    0.106021
apr    0.204787
oct    0.438719
sep    0.449123
dec    0.489011
mar    0.505495
Name: is_positive, dtype: float64

Die Abschlusswahrscheinlichkeit variiert stark zwischen den Kampagnenmonaten.
Dies deutet auf unterschiedliche Kampagnenstrategien sowie ein verändertes
makroökonomisches Umfeld hin.

Der beobachtete Unterschied zwischen Trainings- und Testverteilung ist daher
kein Artefakt des Splits, sondern reflektiert ein realistisches Szenario
zeitlicher Veränderungen (Concept / Prior Shift).

In [38]:
categorical_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
numerical_cols   = X_train.select_dtypes(exclude=["object"]).columns.tolist()

print("Categorical features:", categorical_cols)
print("Numerical features:", numerical_cols)

Categorical features: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'euribor3m']
Numerical features: ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'nr.employed']


### Baseline-Modell: Logistische Regression
Die logistische Regression dient als interpretierbare Baseline zur Einordnung
der Modellperformance.

In [39]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

preprocessor_lr = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), numerical_cols),
    ]
)

logreg_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor_lr),
        ("model", LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            random_state=RANDOM_STATE,
            solver="lbfgs"
        ))
    ]
)

In [40]:
from sklearn.metrics import roc_auc_score, average_precision_score

for C in [0.1, 1.0, 3.0, 10.0]:
    logreg = LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        random_state=RANDOM_STATE,
        solver="lbfgs",
        C=C
    )
    pipe = Pipeline([("preprocessor", preprocessor_lr), ("model", logreg)])
    pipe.fit(X_train, y_train)

    y_proba = pipe.predict_proba(X_test)[:, 1]
    roc = roc_auc_score((y_test == POSITIVE_CLASS).astype(int), y_proba)
    pr  = average_precision_score((y_test == POSITIVE_CLASS).astype(int), y_proba)
    print(f"C={C:<4} | ROC-AUC={roc:.3f} | PR-AUC={pr:.3f} | n_iter={pipe.named_steps['model'].n_iter_}")


C=0.1  | ROC-AUC=0.752 | PR-AUC=0.525 | n_iter=[73]
C=1.0  | ROC-AUC=0.744 | PR-AUC=0.500 | n_iter=[137]
C=3.0  | ROC-AUC=0.743 | PR-AUC=0.499 | n_iter=[176]
C=10.0 | ROC-AUC=0.742 | PR-AUC=0.497 | n_iter=[183]


**Interpretation der Evaluationsmetriken:**  

Die logistische Regression mit stärkerer Regularisierung (C = 0.1) liefert eine
stabile und konvergente Baseline (n_iter_ = 73 << max_iter). 

- **ROC-AUC** misst, wie gut das Modell Kunden mit Abschlusswahrscheinlichkeit
  insgesamt höher einordnet als Kunden ohne Abschluss – unabhängig von einem
  konkreten Schwellenwert. Ein Wert von **0.75** bedeutet, dass das Modell in
  etwa 75 % der Fälle einen zufällig gewählten Kunden mit Abschluss höher
  bewertet als einen Kunden ohne Abschluss.

- **PR-AUC** fokussiert auf die Qualität der Vorhersagen für die positive Klasse
  („Abschluss“) und ist insbesondere bei unausgeglichenen Zielvariablen
  aussagekräftig. Eine PR-AUC von **0.53** liegt deutlich über der Basisrate von 
  rund **31 %** im Testdatensatz und zeigt, dass das Modell relevante Abschlüsse 
  im betrachteten Zeitraum deutlich besser identifiziert als eine zufällige Auswahl.

Zusammengefasst zeigt das Modell eine gute Fähigkeit, Kunden nach ihrer
Abschlusswahrscheinlichkeit zu priorisieren, was für den Einsatz in
Marketingkampagnen entscheidend ist.

In [41]:
# Finales Baseline-Modell (C = 0.1)
logreg_baseline = LogisticRegression(
    max_iter=2000,
    class_weight="balanced",
    random_state=RANDOM_STATE,
    solver="lbfgs",
    C=0.1
)

baseline_pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor_lr),
        ("model", logreg_baseline)
    ]
)

baseline_pipe.fit(X_train, y_train)

# Scores aus genau diesem Modell
y_proba_lr = baseline_pipe.predict_proba(X_test)[:, 1]

In [42]:
def precision_at_k(y_true, y_scores, k=0.1):
    n_top = int(len(y_scores) * k)
    top_idx = np.argsort(y_scores)[-n_top:]
    return (y_true.iloc[top_idx] == POSITIVE_CLASS).mean()

roc_lr = roc_auc_score((y_test == POSITIVE_CLASS).astype(int), y_proba_lr)
pr_lr  = average_precision_score((y_test == POSITIVE_CLASS).astype(int), y_proba_lr)
p10_lr = precision_at_k(y_test.reset_index(drop=True), y_proba_lr, k=0.1)

print(f"LogReg (C=0.1) ROC-AUC       : {roc_lr:.3f}")
print(f"LogReg (C=0.1) PR-AUC        : {pr_lr:.3f}")
print(f"LogReg (C=0.1) Precision@10% : {p10_lr:.3f}")

LogReg (C=0.1) ROC-AUC       : 0.752
LogReg (C=0.1) PR-AUC        : 0.525
LogReg (C=0.1) Precision@10% : 0.569


In [43]:
base_rate_test = (y_test == POSITIVE_CLASS).mean()
print(f"Test base rate: {base_rate_test:.3%} | Precision@10%: {p10_lr:.3%}")

Test base rate: 30.833% | Precision@10%: 56.865%


**Einordnung der Ergebnisse:**  
Die durchschnittliche Abschlussquote im Testdatensatz liegt bei **30,8 %**.
Bei den **Top 10 % der vom Modell priorisierten Kunden** beträgt die
Abschlussquote **56,9 %**, was nahezu einer Verdopplung der
Abschlusswahrscheinlichkeit entspricht.

Dies verdeutlicht den erheblichen geschäftlichen Mehrwert einer
datengetriebenen Kundenpriorisierung im Vergleich zu einer
undifferenzierten Ansprache.

### Performance-Modell: Gradient Boosting (LightGBM)

Zur Erweiterung der Modellanalyse wird ein Gradient-Boosting-Modell (LightGBM)
als leistungsfähiger, nichtlinearer Ansatz eingesetzt. Tree-basierte Modelle
eignen sich insbesondere zur Modellierung nichtlinearer Zusammenhänge sowie
von Interaktionen zwischen Features (z.B. Kampagnenzeitpunkt und
makroökonomisches Umfeld).

Ziel ist es zu prüfen, ob ein solches Modell im Vergleich zur linearen Baseline
zusätzliche Performancegewinne liefert, insbesondere unter zeitlichen
Veränderungen der Datenverteilung.

In [44]:
# Klassenverhältnis im Trainingsdatensatz
n_pos = (y_train == POSITIVE_CLASS).sum()
n_neg = (y_train != POSITIVE_CLASS).sum()

scale_pos_weight = n_neg / n_pos
print(f"scale_pos_weight: {scale_pos_weight:.2f}")

scale_pos_weight: 14.69


Aufgrund der starken Klassenimbalance wird `scale_pos_weight` verwendet, um
Fehler bei der seltenen positiven Klasse („Abschluss“) stärker zu gewichten.

In [45]:
X_train_lgbm = X_train.copy()
X_test_lgbm  = X_test.copy()

for col in categorical_cols:
    X_train_lgbm[col] = X_train_lgbm[col].astype("category")
    # Test ebenfalls category
    X_test_lgbm[col] = X_test_lgbm[col].astype("category")
    # Kategorien im Test exakt wie im Train setzen
    X_test_lgbm[col] = X_test_lgbm[col].cat.set_categories(X_train_lgbm[col].cat.categories)

print("Feature count:", X_train_lgbm.shape[1], X_test_lgbm.shape[1])

Feature count: 19 19


In [46]:
from lightgbm import LGBMClassifier

# Kategorien synchronisieren (wie gehabt)
X_train_lgbm = X_train.copy()
X_test_lgbm  = X_test.copy()

for col in categorical_cols:
    X_train_lgbm[col] = X_train_lgbm[col].astype("category")
    X_test_lgbm[col]  = X_test_lgbm[col].astype("category")
    X_test_lgbm[col]  = X_test_lgbm[col].cat.set_categories(X_train_lgbm[col].cat.categories)

lgbm_native_noes = LGBMClassifier(
    objective="binary",
    n_estimators=1000,
    learning_rate=0.03,
    num_leaves=31,            
    min_child_samples=50,    
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    scale_pos_weight=scale_pos_weight,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

lgbm_native_noes.fit(
    X_train_lgbm,
    (y_train == POSITIVE_CLASS).astype(int),
    categorical_feature=categorical_cols
)

y_proba_lgbm_nat = lgbm_native_noes.predict_proba(X_test_lgbm)[:, 1]

roc_lgbm_nat = roc_auc_score((y_test == POSITIVE_CLASS).astype(int), y_proba_lgbm_nat)
pr_lgbm_nat  = average_precision_score((y_test == POSITIVE_CLASS).astype(int), y_proba_lgbm_nat)
p10_lgbm_nat = precision_at_k(y_test.reset_index(drop=True), y_proba_lgbm_nat, k=0.1)

print(f"LightGBM (native, no ES) ROC-AUC       : {roc_lgbm_nat:.3f}")
print(f"LightGBM (native, no ES) PR-AUC        : {pr_lgbm_nat:.3f}")
print(f"LightGBM (native, no ES) Precision@10% : {p10_lgbm_nat:.3f}")


[LightGBM] [Info] Number of positive: 2100, number of negative: 30850
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001344 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 304
[LightGBM] [Info] Number of data points in the train set: 32950, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.063733 -> initscore=-2.687199
[LightGBM] [Info] Start training from score -2.687199
LightGBM (native, no ES) ROC-AUC       : 0.635
LightGBM (native, no ES) PR-AUC        : 0.439
LightGBM (native, no ES) Precision@10% : 0.529


In [47]:
final_comparison = pd.DataFrame({
    "Model": ["LogReg (C=0.1)", "LightGBM (native, no ES)"],
    "ROC-AUC": [roc_lr, roc_lgbm_nat],
    "PR-AUC": [pr_lr, pr_lgbm_nat],
    "Precision@10%": [p10_lr, p10_lgbm_nat]
})
final_comparison


,Model,ROC-AUC,PR-AUC,Precision@10%
0,LogReg (C=0.1),0.752005,0.524708,0.568651
1,"LightGBM (native, no ES)",0.634763,0.438675,0.528554


Hinweis: „no ES“ bezeichnet ein LightGBM-Modell ohne Early Stopping.
Das Modell wurde bewusst ohne Validierungs-basierten Abbruch trainiert,
um im zeitbasierten Split keine zusätzliche Verzerrung durch einen
internen Validierungszeitraum zu erzeugen.

**Einordnung der Modellvergleiche:**  

Im zeitbasierten Evaluationsszenario zeigt die logistische Regression mit
stärkerer Regularisierung (C = 0.1) über alle betrachteten Metriken hinweg
eine bessere Performance als das Gradient-Boosting-Modell (LightGBM).

Insbesondere die höhere **PR-AUC** und **Precision@10 %** deuten darauf hin,
dass die logistische Regression Kunden im betrachteten Zeitraum zuverlässiger
nach Abschlusswahrscheinlichkeit priorisiert.

Das vergleichsweise schwächere Abschneiden von LightGBM lässt darauf schließen,
dass die zugrunde liegenden Zusammenhänge in diesem Anwendungsfall überwiegend
linear und additiv sind und komplexere nichtlineare Modelle unter starkem
zeitlichem Shift keinen zusätzlichen Mehrwert liefern.

In [48]:
fi_gain = pd.Series(
    lgbm_native_noes.booster_.feature_importance(importance_type="gain"),
    index=X_train_lgbm.columns
).sort_values(ascending=False)

fi_gain_pct = fi_gain / fi_gain.sum() * 100
fi_gain_pct.head(15)


euribor3m         22.428480
age               19.137736
campaign           8.967553
month              7.851756
job                7.601037
education          5.982914
marital            4.127254
housing            3.678791
cons.price.idx     3.383104
day_of_week        3.348935
cons.conf.idx      3.019024
loan               2.336453
default            2.220344
contact            2.014748
emp.var.rate       1.821564
dtype: float64

**Feature Importance (LightGBM):**  
Die Feature-Importance-Analyse bezieht sich ausschließlich auf das
Gradient-Boosting-Modell (LightGBM) und dient der explorativen Einordnung
möglicher nichtlinearer Effekte.

Die Importances konzentrieren sich auf wenige zentrale Treiber wie das
Zinsniveau (`euribor3m`) und kundenbezogene Merkmale (`age`). Hinweise auf 
stark zusätzliche nichtlineare Effekte sind begrenzt.

Dies legt nahe, dass ein großer Teil der Vorhersagekraft bereits durch
einfache additive Zusammenhänge abgebildet werden kann.

### **Modellentscheidung (zeitbasierter Split):** 
Im realistischen, zeitbasierten Evaluationsszenario erzielt die logistische
Regression mit starker Regularisierung (C = 0.1) die beste Performance über
alle relevanten Metriken hinweg.

Insbesondere die höhere PR-AUC sowie die bessere Precision@10 % sprechen
für eine stabilere und verlässlichere Kundenpriorisierung unter zeitlichem
Daten- und Marktshift. Komplexere Modelle wie LightGBM zeigten in diesem
Setting keinen zusätzlichen Performancegewinn.

### Ergänzende Analyse: Stratifizierter Zufallssplit

Neben der zeitbasierten Evaluation wird ergänzend ein stratifizierter
Zufallssplit durchgeführt. Ziel ist es, die grundsätzliche Modellfähigkeit
unter stabilen Datenbedingungen zu analysieren, bei denen Trainings- und
Testdaten dieselbe Zielverteilung aufweisen.

Diese Analyse dient ausschließlich der methodischen Einordnung und stellt
kein realistisches Szenario für den Produktiveinsatz dar, da zeitliche
Veränderungen von Kampagnen- und Marktbedingungen bewusst ausgeblendet werden.

In [49]:
from sklearn.model_selection import train_test_split

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("Train positive rate:", (y_train_s == POSITIVE_CLASS).mean())
print("Test positive rate :", (y_test_s == POSITIVE_CLASS).mean())

Train positive rate: 0.11265553869499241
Test positive rate : 0.11264870114105366


In [50]:
# LogReg Baseline auf stratified split
baseline_pipe_s = Pipeline(
    steps=[
        ("preprocessor", preprocessor_lr),
        ("model", LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            random_state=RANDOM_STATE,
            solver="lbfgs",
            C=0.1
        ))
    ]
)

baseline_pipe_s.fit(X_train_s, y_train_s)

y_proba_lr_s = baseline_pipe_s.predict_proba(X_test_s)[:, 1]

roc_lr_s = roc_auc_score((y_test_s == POSITIVE_CLASS).astype(int), y_proba_lr_s)
pr_lr_s  = average_precision_score((y_test_s == POSITIVE_CLASS).astype(int), y_proba_lr_s)
p10_lr_s = precision_at_k(y_test_s.reset_index(drop=True), y_proba_lr_s, k=0.1)

print(f"LogReg (strat) ROC-AUC       : {roc_lr_s:.3f}")
print(f"LogReg (strat) PR-AUC        : {pr_lr_s:.3f}")
print(f"LogReg (strat) Precision@10% : {p10_lr_s:.3f}")

LogReg (strat) ROC-AUC       : 0.808
LogReg (strat) PR-AUC        : 0.474
LogReg (strat) Precision@10% : 0.521


In [51]:
X_train_s_lgbm = X_train_s.copy()
X_test_s_lgbm  = X_test_s.copy()

for col in categorical_cols:
    X_train_s_lgbm[col] = X_train_s_lgbm[col].astype("category")
    X_test_s_lgbm[col]  = X_test_s_lgbm[col].astype("category")
    X_test_s_lgbm[col]  = X_test_s_lgbm[col].cat.set_categories(
        X_train_s_lgbm[col].cat.categories
    )

print("Feature count:", X_train_s_lgbm.shape[1])

Feature count: 19


In [52]:
# Klassenverhältnis neu berechnen (stratified train!)
n_pos_s = (y_train_s == POSITIVE_CLASS).sum()
n_neg_s = (y_train_s != POSITIVE_CLASS).sum()
scale_pos_weight_s = n_neg_s / n_pos_s

lgbm_s = LGBMClassifier(
    objective="binary",
    n_estimators=1000, 
    learning_rate=0.03, 
    num_leaves=31, 
    min_child_samples=50, 
    subsample=0.8, 
    colsample_bytree=0.8, 
    scale_pos_weight=scale_pos_weight_s,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

lgbm_s.fit(
    X_train_s_lgbm,
    (y_train_s == POSITIVE_CLASS).astype(int),
    categorical_feature=categorical_cols
)

[LightGBM] [Info] Number of positive: 3712, number of negative: 29238
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001202 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 531
[LightGBM] [Info] Number of data points in the train set: 32950, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.112656 -> initscore=-2.063898
[LightGBM] [Info] Start training from score -2.063898


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.03
,n_estimators,1000
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,50


In [53]:
y_proba_lgbm_s = lgbm_s.predict_proba(X_test_s_lgbm)[:, 1]

roc_lgbm_s = roc_auc_score((y_test_s == POSITIVE_CLASS).astype(int), y_proba_lgbm_s)
pr_lgbm_s  = average_precision_score((y_test_s == POSITIVE_CLASS).astype(int), y_proba_lgbm_s)
p10_lgbm_s = precision_at_k(y_test_s.reset_index(drop=True), y_proba_lgbm_s, k=0.1)

print(f"LightGBM (strat) ROC-AUC       : {roc_lgbm_s:.3f}")
print(f"LightGBM (strat) PR-AUC        : {pr_lgbm_s:.3f}")
print(f"LightGBM (strat) Precision@10% : {p10_lgbm_s:.3f}")

LightGBM (strat) ROC-AUC       : 0.796
LightGBM (strat) PR-AUC        : 0.475
LightGBM (strat) Precision@10% : 0.530


In [54]:
comparison_strat_df = pd.DataFrame({
    "Model": ["LogReg (C=0.1)", "LightGBM"],
    "ROC-AUC": [roc_lr_s, roc_lgbm_s],
    "PR-AUC": [pr_lr_s, pr_lgbm_s],
    "Precision@10%": [p10_lr_s, p10_lgbm_s]
})

comparison_strat_df

,Model,ROC-AUC,PR-AUC,Precision@10%
0,LogReg (C=0.1),0.807891,0.473538,0.521264
1,LightGBM,0.796025,0.474618,0.529769


**Bewertung der Splitting-Strategien:**  

Der stratifizierte Zufallssplit erlaubt einen fairen Vergleich der reinen
Modellkapazität unter stabilen Datenbedingungen, da Trainings- und Testdaten
dieselbe Zielverteilung aufweisen.

In diesem Setting zeigen beide Modelle eine vergleichbare Gesamtperformance.
Während die logistische Regression bei ROC-AUC leicht bessere Werte erzielt, 
weist das LightGBM-Modell eine minimal höhere PR-AUC und Precision@10% auf.

Dies deutet darauf hin, dass nichtlineare Modelle in spezifischen operativen
Szenarien (z.B. strikte Top-K-Priorisierung unter stabilen Bedingungen)
Vorteile bieten können, ohne jedoch insgesamt eine überlegene Modellperformance
zu liefern.

Für den realistischen Produktiveinsatz ist der zeitbasierte Split jedoch
aussagekräftiger, da er Veränderungen der Kampagnenstrategie und des
makroökonomischen Umfelds berücksichtigt. In diesem Szenario erweist sich die
logistische Regression als robuster Ansatz mit stabilerer Generalisierung.

## Management Summary

**Kernaussage:**  
Die Analyse zeigt, dass eine datengetriebene Priorisierung von Kundenkontakten
die Effizienz von Festgeldkampagnen signifikant steigern kann.

Im realistischen, zeitbasierten Evaluationsszenario erzielt eine logistische
Regression mit starker Regularisierung (C = 0.1) die stabilste Performance.
Bei den **Top 10 % der vom Modell priorisierten Kunden** liegt die Abschlussquote
bei rund **57 %**, gegenüber einer durchschnittlichen Quote von rund **31 %**
im Testzeitraum.

### Beispielrechnung (vereinfachte Illustration)
Angenommen, es können **10.000 Kunden** aus einem Gesamtpool von **100.000**
telefonisch kontaktiert werden:

- **Ohne Modell (zufällige Auswahl):**
  - Abschlussquote ~31 % → ca. **3.100 Abschlüsse**
- **Mit Modellpriorisierung (Top 10 %):**
  - Abschlussquote ~57 % → ca. **5.700 Abschlüsse**

**+2.600 zusätzliche Abschlüsse bei gleichem Kampagnenbudget**

**Empfehlung:**  
Einsatz des Modells als **Scoring-Mechanismus zur Priorisierung von Kundenkontakten**
sowie kontinuierliches Monitoring (PR-AUC, Precision@Top-K) und regelmäßiges Retraining.

## Weiterführende Fragestellungen

### Potenzial zusätzlicher Datenquellen
- **Kundenbeziehungshistorie** (z.B. Dauer der Kundenbeziehung, frühere Abschlüsse/Kampagnenreaktionen, Interaktionshistorie)
- **Produktnutzung & Ereignisse** (z.B. aktive Produkte, Nutzungshäufigkeit, Kontoaktivität, „Life Events“ wie Gehaltseingänge/Umzüge)
- **Transaktions- und Kontodaten** (z.B. Zahlungsströme, Spar-/Liquiditätsindikatoren, Saldoentwicklung, Gehaltseingang)
- **Recency/Frequency/Monetary-Features (RFM)** (z.B. letzte Produktaktivität, Kontaktfrequenz, durchschnittliche Einzahlungs-/Sparvolumina)
- **Angebots- und Preisinformationen** (z.B. angebotener Zinssatz vs. Marktzinssatz/Spread, Laufzeit, Mindestanlage, Konditionen)
- **Kampagnen- und Kontaktmetadaten** (z.B. Kanal, Uhrzeit, Ansprechpartner, Touchpoint-Sequenz, Kontaktqualität)
- **Feedback-/Outcome-Daten** (z.B. Ablehnungsgründe, Follow-up-Interesse, Terminvereinbarungen, Beschwerde-/Satisfaction-Signale)

### Übertragbarkeit auf andere Bankbereiche
Der methodische Ansatz ist auf weitere bankfachliche Anwendungsfälle übertragbar,
bei denen eine gezielte Priorisierung von Kundenkontakten erforderlich ist.
Beispiele hierfür sind:

- **Cross-Selling** (z. B. Kreditkarten, Konsumentenkredite, Wertpapierdepots):
  Priorisierung von Kunden mit hoher Abschlusswahrscheinlichkeit für ein Zusatzprodukt.

- **Churn-Prävention**:
  Identifikation von Kunden mit erhöhtem Abwanderungsrisiko, um proaktive
  Retentionsmaßnahmen gezielt einzusetzen.

- **Next-Best-Action / Next-Best-Offer**:
  Auswahl der jeweils sinnvollsten Maßnahme oder Produktansprache je Kunde
  auf Basis von Kundenprofil, Historie und aktuellen Marktbedingungen.

- **Kampagnensteuerung über mehrere Kanäle**:
  Erweiterung des Ansatzes auf Omnichannel-Szenarien (Telefon, E-Mail, Online-Banking),
  wobei das Modell zusätzlich den optimalen Kontaktkanal vorhersagen kann.

In allen Fällen bleibt der grundlegende Modellierungsprozess identisch:
Die Zielvariable wird angepasst, während Feature-Engineering, Training,
Evaluation und Scoring analog durchgeführt werden.